In [5]:
from anthropic import AnthropicVertex
from os import environ

region = environ.get("CLOUD_ML_REGION", "")
project_id = environ.get("ANTHROPIC_VERTEX_PROJECT_ID", "")
client = AnthropicVertex(region=region, project_id=project_id)
model = "claude-sonnet-4-5@20250929"

In [6]:
# Helper functions
from typing import List, Optional


def add_user_message(messages: List[dict], content: str):
    _add_role_message(messages, "user", content)


def add_agent_message(messages: List[dict], content: str):
    _add_role_message(messages, "assistant", content)


def _add_role_message(messages: List[dict], role: str, content: str):
    messages.append({"role": role, "content": content})


""" Sends a chat request to the Claude model with the given messages. """


def chat(
    messages: List[dict],
    system: Optional[str] = None,
    temperature: float = 1.0,
    stop_sequences: List[str] = None,
) -> str:
    params = {
        "model": model,
        "max_tokens": 1000,
        "messages": messages,
        "temperature": temperature,
        "stop_sequences": stop_sequences,
    }

    if system is not None:
        params["system"] = system

    response = client.messages.create(**params)
    return response.content[0].text

In [7]:
# Function to generate a new dataset
import json


def generate_dataset():
    prompt = """
Generate a evaluation dataset for a prompt evaluation. The dataset will be used to evaluate prompts
that generate Python, JSON, or Regex specifically for AWS-related tasks. Generate an array of JSON objects,
each representing task that requires Python, JSON, or a Regex to complete.

Example output:
```json
[
    {
        "task": "Description of task",
        "format": "json" or "python" or "regex",
        "solution_criteria": "Key criteria for evaluating the solution"
    },
    ...additional
]
```

* Focus on tasks that can be solved by writing a single Python function, a single JSON object, or a regular expression.
* Focus on tasks that do not require writing much code

Please generate 3 objects.
"""

    messages = []
    add_user_message(messages, prompt)
    add_agent_message(messages, "```json")
    text = chat(messages, stop_sequences=["```"])
    return json.loads(text)

In [8]:
# Generate the dataset and write it to 'dataset.json'
dataset = generate_dataset()
with open("dataset.json", "w") as f:
    json.dump(dataset, f, indent=2)

In [9]:
# Function to grade a test case + output using a model
def grade_by_model(test_case, output):
    eval_prompt = f"""
You are an expert AWS code reviewer. Your task is to evaluate the following AI-generated solution.

Original Task:
<task>
{test_case["task"]}
</task>

Solution to Evaluate:
<solution>
{output}
</solution>

Criteria you should use to evaluate the solution:
<criteria>
{test_case["solution_criteria"]}
</criteria>

Output Format
Provide your evaluation as a structured JSON object with the following fields, in this specific order:
- "strengths": An array of 1-3 key strengths
- "weaknesses": An array of 1-3 key areas for improvement
- "reasoning": A concise explanation of your overall assessment
- "score": A number between 1-10

Respond with JSON. Keep your response concise and direct.
Example response shape:
{{
    "strengths": string[],
    "weaknesses": string[],
    "reasoning": string,
    "score": number
}}
    """

    messages = []
    add_user_message(messages, eval_prompt)
    add_agent_message(messages, "```json")
    eval_text = chat(messages, stop_sequences=["```"])
    return json.loads(eval_text)

In [10]:
# Passes a test case into Claude
def run_prompt(test_case):
    prompt = f"""
Please solve the following task:

{test_case["task"]}

* Respond only with Python, JSON, or a plain Regex
* Do not add any comments or commentary or explanation
"""

    messages = []
    add_user_message(messages, prompt)
    add_agent_message(messages, "```code")
    output = chat(messages, stop_sequences=["```"])
    return output

In [11]:
# Functions to validate the output structure
import re
import ast


def validate_json(text):
    try:
        json.loads(text.strip())
        return 10
    except json.JSONDecodeError:
        return 0


def validate_python(text):
    try:
        ast.parse(text.strip())
        return 10
    except SyntaxError:
        return 0


def validate_regex(text):
    try:
        re.compile(text.strip())
        return 10
    except re.error:
        return 0


def grade_syntax(response, test_case):
    format = test_case["format"]
    if format == "json":
        return validate_json(response)
    elif format == "python":
        return validate_python(response)
    else:
        return validate_regex(response)

In [12]:
# Function to execute a single test case and grade the output
def run_test_case(test_case):
    """Calls run_prompt, then grades the result"""
    output = run_prompt(test_case)

    model_grade = grade_by_model(test_case, output)
    model_score = model_grade["score"]
    reasoning = model_grade["reasoning"]

    syntax_score = grade_syntax(output, test_case)

    score = (model_score + syntax_score) / 2

    return {
        "output": output,
        "test_case": test_case,
        "score": score,
        "reasoning": reasoning,
    }

In [13]:
from statistics import mean


def run_eval(dataset):
    """Loads the dataset and calls run_test_case with each case"""
    results = []

    for test_case in dataset:
        result = run_test_case(test_case)
        results.append(result)

    average_score = mean([result["score"] for result in results])
    print(f"Average score: {average_score}")

    return results

In [14]:
with open("dataset.json", "r") as f:
    dataset = json.load(f)

results = run_eval(dataset)

Average score: 8.333333333333334


In [15]:
print(json.dumps(results, indent=2))

[
  {
    "output": "\n^(?!.*\\.\\.)(?!.*\\.\\-)(?!.*\\-\\.)[a-z0-9][a-z0-9.-]{1,61}[a-z0-9]$\n",
    "test_case": {
      "task": "Write a regular expression that validates AWS S3 bucket names according to AWS naming rules: 3-63 characters long, only lowercase letters, numbers, hyphens, and periods, must start and end with a letter or number, and cannot have consecutive periods or period-hyphen combinations.",
      "format": "regex",
      "solution_criteria": "Regex must match valid S3 bucket names (e.g., 'my-bucket-123', 'example.bucket') and reject invalid ones (e.g., 'My-Bucket', 'bucket..name', '-invalid', 'ab'). Should enforce all AWS S3 naming constraints."
    },
    "score": 8.5,
    "reasoning": "The regex handles most basic S3 naming rules well, including character restrictions, start/end requirements, and preventing problematic character combinations. However, it has a subtle length issue and misses two important AWS-specific restrictions: IP address formats and the 'xn--